In [107]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from deepctr.models import DeepFM,DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
import numpy as np
import tensorflow as tf
import random
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variable(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r
gid_dict = load_variable('/home/jovyan/dataset/gid_dict')#所有gid對應編號的dict
df503 = load_variable('/home/jovyan/dataset/original_candidate_itemset/0503_candidate_itemset')#舊方法的candidate_itemset


### start training(use 4/1-5/2 model train5/3 data)

In [42]:
df_uid_5 = load_variable('/home/jovyan/dataset/may_df_uid')
df_ven_guid_5 = load_variable('/home/jovyan/dataset/may_df_ven_guid')
df_uid53 = df_uid_5[df_uid_5.month==53]
df_ven_guid53 = df_ven_guid_5[df_ven_guid_5.month==53]

In [43]:
df_temp = pd.concat([df_uid53[['month','gid_encode']],df_ven_guid53[['month','gid_encode']]],ignore_index=True)  
under = 0
freq_list = []
freq = df_temp.gid_encode.value_counts()
for i in freq.values:
    under += np.log10(i+1)
for i in freq.values:
    freq_list.append(np.log10(i+1)/under)

In [44]:
import random
def create_train_set(df,bywhich):
    t_set = []
    neg_list = random.choices(freq.index,weights = freq_list, k=1000000)
    for userid, hist in df.groupby(bywhich):
        pos_list = hist['gid_encode'].tolist()
        if len(pos_list) > 101:
            pos_list = pos_list[-101:]
            print(userid)
        def gen_neg():
            count = 0
            neg = pos_list[0]
            while neg in pos_list:
                if count == 100:
                    count = 0
                neg = random.choices(neg_list, k=1)[0]
                count += 1
            return neg
        neg_list2 = [gen_neg() for i in range(len(pos_list))]
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] 
            t_set.append((userid, hist, pos_list[i], 1))
            t_set.append((userid, hist, neg_list2[i], 0))
    return t_set
uid_set = create_train_set(df_uid53,'uid')
ven_guid_set = create_train_set(df_ven_guid53,'ven_guid')

12765
13708
15577
33136
38560
39859
42462
46878
47911
50237
54573
59573
61991
63463
68141
73091
79456
79880
82133
84280
89271
99483
103277
121446
125908
208203
351989
426698
574791
647826


In [45]:
train_set503 = uid_set+ven_guid_set
np.random.shuffle(train_set503)
len(uid_set),len(ven_guid_set)

(198754, 221184)

In [46]:
def create_array(data):
    g_id = []
    hist_g_id = []
    click = []
    seq_length = []

    for index, value in enumerate(data):
        hist_g_id.append(value[1])
        seq_length.append(len(value[1]))
        g_id.append(value[2])
        click.append(value[3])  
    for i in range(len(hist_g_id)):
        if len(hist_g_id[i]) != 100:
            hist_g_id[i] += (100-len(hist_g_id[i]))*[0]

    g_id = np.array(g_id)
    hist_g_id = np.array(hist_g_id)
    click = np.array(click)
    seq_length = np.array(seq_length)
    return g_id, hist_g_id, click, seq_length
g_id_train503, hist_g_id_train503, click_train503, seq_length_train503 = create_array(train_set503)

In [49]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', 1870599, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=1870599, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_length')]

x = {'g_id': g_id_train503, 
    'hist_g_id': hist_g_id_train503, 
    'seq_length': seq_length_train503}
y = click_train503

In [40]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_502")
model.load_weights(latest)

checkpoint_path = "/home/jovyan/dataset/checkpoint_folder/model_401_503/rakuten_0401_0503.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
history401_503 = model.fit(x, y, batch_size=128, verbose=1, epochs=10, callbacks=[cp_callback])

Epoch 1/10
3281/3281 [==============================] - 875s 266ms/step - loss: 0.5653 - accuracy: 0.7094 - auc: 0.7806

Epoch 00001: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_503/rakuten_0401_0503.ckpt
Epoch 2/10
3281/3281 [==============================] - 878s 268ms/step - loss: 0.4733 - accuracy: 0.7750 - auc: 0.8537

Epoch 00002: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_503/rakuten_0401_0503.ckpt
Epoch 3/10
3281/3281 [==============================] - 859s 262ms/step - loss: 0.3947 - accuracy: 0.8257 - auc: 0.9013

Epoch 00003: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_503/rakuten_0401_0503.ckpt
Epoch 4/10
3281/3281 [==============================] - 851s 259ms/step - loss: 0.3255 - accuracy: 0.8652 - auc: 0.9335

Epoch 00004: saving model to /home/jovyan/dataset/checkpoint_folder/model_401_503/rakuten_0401_0503.ckpt
Epoch 5/10
3281/3281 [==============================] - 911s 278ms/step - loss: 0.2759 - accurac

### 15個epoch

In [50]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/checkpoint_folder/model_401_503")
model.load_weights(latest)

checkpoint_path =  "/home/jovyan/dataset/15epoch/model401_503/401_503.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
history15epoch = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
3281/3281 [==============================] - 948s 281ms/step - loss: 0.3979 - accuracy: 0.8310 - auc: 0.9032

Epoch 00001: saving model to /home/jovyan/dataset/15epoch/model401_503/401_503.ckpt
Epoch 2/5
3281/3281 [==============================] - 930s 283ms/step - loss: 0.3076 - accuracy: 0.8777 - auc: 0.9432

Epoch 00002: saving model to /home/jovyan/dataset/15epoch/model401_503/401_503.ckpt
Epoch 3/5
3281/3281 [==============================] - 960s 293ms/step - loss: 0.2468 - accuracy: 0.9081 - auc: 0.9635

Epoch 00003: saving model to /home/jovyan/dataset/15epoch/model401_503/401_503.ckpt
Epoch 4/5
3281/3281 [==============================] - 915s 279ms/step - loss: 0.2012 - accuracy: 0.9280 - auc: 0.9754

Epoch 00004: saving model to /home/jovyan/dataset/15epoch/model401_503/401_503.ckpt
Epoch 5/5
3281/3281 [==============================] - 859s 262ms/step - loss: 0.1677 - accuracy: 0.9425 - auc: 0.9824

Epoch 00005: saving model to /home/jovyan/dataset/15epoch/model4

### 20 epoch

In [52]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
latest = tf.train.latest_checkpoint("/home/jovyan/dataset/15epoch/model401_503")
model.load_weights(latest)
checkpoint_path = "/home/jovyan/dataset/20epoch/model401_503/401_503.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
histtory5epoch = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
3281/3281 [==============================] - 920s 279ms/step - loss: 0.1435 - accuracy: 0.9526 - auc: 0.9869

Epoch 00001: saving model to /home/jovyan/dataset/20epoch/model401_503/401_503.ckpt
Epoch 2/5
3281/3281 [==============================] - 918s 280ms/step - loss: 0.1275 - accuracy: 0.9597 - auc: 0.9892

Epoch 00002: saving model to /home/jovyan/dataset/20epoch/model401_503/401_503.ckpt
Epoch 3/5
3281/3281 [==============================] - 921s 281ms/step - loss: 0.1159 - accuracy: 0.9643 - auc: 0.9908

Epoch 00003: saving model to /home/jovyan/dataset/20epoch/model401_503/401_503.ckpt
Epoch 4/5
3281/3281 [==============================] - 887s 270ms/step - loss: 0.1041 - accuracy: 0.9689 - auc: 0.9923

Epoch 00004: saving model to /home/jovyan/dataset/20epoch/model401_503/401_503.ckpt
Epoch 5/5
3281/3281 [==============================] - 825s 252ms/step - loss: 0.0966 - accuracy: 0.9721 - auc: 0.9931

Epoch 00005: saving model to /home/jovyan/dataset/20epoch/model4

### make request

In [ ]:
def clean_candidate(df):#舊方法刪完後的結果(不存在就刪掉)
    df = df[df['gid'].isin(list(gid_dict.keys()))]
    df = df.reset_index(drop=True)
    df.candidate_items = df.candidate_items.apply(lambda x:x.split(','))
    df = df.explode('candidate_items', ignore_index=True)
    df = df[df['candidate_items'].isin(list(gid_dict.keys()))]#推薦清單 candidate_item清掉不在4/1-5/5的商品
    df.gid = df.gid.apply(lambda x:gid_dict[x])
    df.candidate_items = df.candidate_items.apply(lambda x:gid_dict[x])
    df = df.groupby(['gid']).agg({'candidate_items': lambda x: x.tolist()})
    df = df.reset_index()
    return df
df_candidate = clean_candidate(df503)

In [ ]:
import requests
import json
API_ENDPOINT = "http://localhost:5000/"
x_request = {
            'hist_g_id':hist_g_id_train503[:1].tolist()[0],
            'candidate':df_candidate[df_candidate.gid == 751602]['candidate_items'].tolist()[0],
            'seq_length':seq_length_train503.tolist()[0]
            }
r = requests.post(url=API_ENDPOINT, json=x_request)
print(r.text)
#r = r.json()
#print(r['rerank'])

In [117]:
for i,v in x_request.items():
    print(i,v)

hist_g_id [1315304, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
candidate [1046873, 798421, 990655, 226697, 23467, 1333060, 1030811, 94341, 1539411, 1534230, 757623, 1519174, 1839503, 898817, 223846, 1598160, 555647, 1846256, 232675, 1264920, 686272, 786385, 1028995, 1313552, 1076777, 1029365, 368416, 531902, 1326280, 436448]
seq_length 1
